In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [ ]:
# Lee la base de urgencias
ruta_urgencias = "../data/raw/9_urgencias/df_procesada_urgencia.csv"
df = pd.read_csv(ruta_urgencias, dtype={"id_paciente": str})

C:\Users\jarojas\AppData\Local\Temp\ipykernel_18660\3913763877.py:1: DtypeWarning: Columns (16,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/9_urgencias/df_procesada_urgencia.csv", dtype={"id_paciente": str})
